#### Mục tiêu: Xây dựng mô hình có độ chính xác trên tập validation khoảng 87%

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

In [ ]:

import tensorflow as tf

import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteINUCCE/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteINUCCE/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteINUCCE/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
list(train_data.take(1))[0]

(<tf.Tensor: shape=(163,), dtype=int64, numpy=
 array([  62,   18,   41,  604,  927,   65,    3,  644, 7968,   21,   35,
        5096,   36,   11,   43, 2948, 5240,  102,   50,  681, 7862, 1244,
           3, 3266,   29,  122,  640,    2,   26,   14,  279,  438,   35,
          79,  349,  384,   11, 1991,    3,  492,   79,  122,  188,  117,
          33, 4047, 4531,   14,   65, 7968,    8, 1819, 3947,    3,   62,
          27,    9,   41,  577, 5044, 2629, 2552, 7193, 7961, 3642,    3,
          19,  107, 3903,  225,   85,  198,   72,    1, 1512,  738, 2347,
         102, 6245,    8,   85,  308,   79, 6936, 7961,   23, 4981, 8044,
           3, 6429, 7961, 1141, 1335, 1848, 4848,   55, 3601, 4217, 8050,
           2,    5,   59, 3831, 1484, 8040, 7974,  174, 5773,   22, 5240,
         102,   18,  247,   26,    4, 3903, 1612, 3902,  291,   11,    4,
          27,   13,   18, 4092, 4008, 7961,    6,  119,  213, 2774,    3,
          12,  258, 2306,   13,   91,   29,  171,   52,  229,    

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# Tạo một tokenizer mới
tokenizer = Tokenizer(num_words=25000, oov_token='<OOV>')

# Lấy dữ liệu text từ 'train_data' và chuyển thành list
texts = [text.numpy().decode('utf-8') for text, _ in train_data]

# Fit tokenizer trên dữ liệu text
tokenizer.fit_on_texts(texts)


AttributeError: 'numpy.ndarray' object has no attribute 'decode'

In [ ]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

train_dataset = train_data.shuffle(BUFFER_SIZE)
padded_shapes = tuple(spec.shape if spec.shape.rank is not None else tf.TensorShape(None) for spec in train_data.element_spec)
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)


In [ ]:
for item in train_data:
  print(item)

(<tf.Tensor: shape=(64, 1109), dtype=int64, numpy=
array([[  62,   18,   41, ...,    0,    0,    0],
       [  12,   31,   93, ...,    0,    0,    0],
       [ 636,  102, 4714, ...,    0,    0,    0],
       ...,
       [ 133, 1955,    5, ...,    0,    0,    0],
       [2635, 1610,   48, ...,    0,    0,    0],
       [  12,  196, 7968, ...,    0,    0,    0]])>, <tf.Tensor: shape=(64,), dtype=int64, numpy=
array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])>)
(<tf.Tensor: shape=(64, 1179), dtype=int64, numpy=
array([[3943,    4,  848, ...,    0,    0,    0],
       [1646, 1760,  463, ...,    0,    0,    0],
       [1993,   38,  601, ...,    0,    0,    0],
       ...,
       [  12,   81,  196, ...,    0,    0,    0],
       [3404, 5426, 7961, ...,    0,    0,    0],
       [ 156,    1, 4030, ...,    0,    0,    0]])>, <tf.Ten

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, RNN, Dense, LSTM, Bidirectional, SimpleRNN
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
import tensorflow as tf

# Xây dựng model

# FIXME: Tạo model Sequential

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(25000, 64, input_length= 200),
    tf.keras.layers.GlobalAveragePooling1D(), # Ta hoàn toàn có thể sử dụng một lớp Convolution 1D sau embedding để tìm mối quan hệ giữa các từ
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
num_epochs = 10

# FIXME: Compile Optimizer và Loss function

model.compile(optimizer=Adam(learning_rate = 0.001), loss='binary_crossentropy', metrics=['acc'])

# FIXME: Tiến hành training. Mục tiêu: 87 %


history = model.fit(train_data, epochs = num_epochs, validation_data = test_data)

Epoch 1/10
391/391 [==============================] - 115s 151ms/step - loss: 0.6644 - acc: 0.6326 - val_loss: 0.5852 - val_acc: 0.7328
Epoch 2/10
391/391 [==============================] - 5s 13ms/step - loss: 0.4548 - acc: 0.8239 - val_loss: 0.3867 - val_acc: 0.8537
Epoch 3/10
391/391 [==============================] - 4s 10ms/step - loss: 0.3187 - acc: 0.8840 - val_loss: 0.3304 - val_acc: 0.8724
Epoch 4/10
391/391 [==============================] - 3s 9ms/step - loss: 0.2638 - acc: 0.9050 - val_loss: 0.3128 - val_acc: 0.8768
Epoch 5/10
391/391 [==============================] - 4s 11ms/step - loss: 0.2314 - acc: 0.9152 - val_loss: 0.3083 - val_acc: 0.8766
Epoch 6/10
391/391 [==============================] - 3s 9ms/step - loss: 0.2085 - acc: 0.9248 - val_loss: 0.3112 - val_acc: 0.8758
Epoch 7/10
391/391 [==============================] - 4s 10ms/step - loss: 0.1907 - acc: 0.9312 - val_loss: 0.3189 - val_acc: 0.8753
Epoch 8/10
391/391 [==============================] - 4s 11ms/step -